## Visualize a Monte Carlo

This script plots Monte Carlo simulated electron trajectories.

In [1]:
using NeXLCore
using MeshCat, GeometryBasics, Colors
using CoordinateTransformations

vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat C:\Users\nicho\.julia\packages\MeshCat\C1FSW\src\visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8700))

In [2]:
# Draw the sample regions
function draw(vis::Visualizer, region::Region, colors::Dict{Material, Color}, name::String="Sample")
    for (i, ch) in enumerate(region.children)
        chname = "$name[$i]"
        setobject!(vis[chname], MeshObject(ch.shape, MeshBasicMaterial(color=RGBA(colors[ch.material],0.2f0))))
        settransform!(vis[chname],LinearMap([ 1.0e4 0.0 0.0; 0.0 1.0e4 0.0; 0.0 0.0 -1.0e4]))
        draw(vis, ch, colors, chname)
    end
end

draw (generic function with 2 methods)

In [4]:
# Clear the display
delete!(vis)
# Construct a new sample
p = coated_particle(material("Al",1.9), 1.0e-4, material("Al2O3",1.9), 1.0e-5, material("C",1.0))
# Select colors
matcolors = colorize(p)
# Run many trajectories
draw(vis, p, matcolors, "Particle")
allpts, allcolors = Vector{Position}[], Vector{RGB{Float32}}[]
# Display each trajectory alone
for i in 1:100
    pts, colors=Position[], RGB{Float32}[]
    trajectory(gun(Electron, 20.0e3, 1.0e-6), p) do part, reg
        # Record the position and the color at each scatter point
        push!(pts, NeXLCore.position(part) .* (1.0e4, 1.0e4, -1.0e4))
        push!(colors, matcolors[reg.material])
    end
    # Add the list of points to the display
    setobject!(vis["trajectory"], PointCloud(pts, colors)) # Add replacements
    sleep(0.1)
    delete!(vis["trajectory"]) # Remove previous trajectories
    push!(allpts,pts)
    push!(allcolors,colors)
end
# Display all the trajectories simultaneously
for (i, (pts, colors)) in enumerate(zip(allpts, allcolors))
    setobject!(vis["trajectory[$i]"], PointCloud(pts, colors))
end